In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Cristovao Beirao da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from fnmatch import fnmatch
import scipy.stats as stats
import hist
import mplhep as hep
plt.style.use(hep.style.CMS)
from notebooks.notebook_helpers import *

from scripts.log_action import log_action_v2
from time import sleep

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_EPIR_1_1"]
# chip_names = ["ET2-W36-IP5-20", "ET2-W36-IP7-13", "ET2-W36-IP7-10"]
# old 
# chip_names = ["ET2-W36-IP7-12", "ET2-W36-IP5-15", "ET2-W36-IP5-14"]
# old [0x78, 0x61, 0x74]
# old top modified C type =  ET2-W36-IP5-15
# replaced with new type D = ET2-W36-IP7-10
chip_fignames = chip_names
chip_figtitles = ["ET2_BumpBond_EPIR_1_1"]

# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60]
# chip_addresses = [0x78, 0x74, 0x61]
ws_addresses = [None, None, None]

today = datetime.date.today().isoformat()
# today = "2023-09-14"


# i2c_gui.__no_connect__ = False  # Set to fake connecting to an ETROC2 device
# i2c_gui.__no_connect_type__ = "echo"  # for actually testing readback
# #i2c_gui.__no_connect_type__ = "check"  # default behaviour

hostname = "192.168.2.3"

polarity = "0x000f"

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,chip_names,[("1","1")])
# i2c_conn = i2c_connection(port,chip_addresses,chip_names,[("1","1"), ("1","1"), ("1","1")])

In [ ]:
chip = i2c_conn.get_chip_i2c_connection(0x60)

row_indexer_handle,_,_ = chip.get_indexer("row")
column_indexer_handle,_,_ = chip.get_indexer("column")
broadcast_handle,_,_ = chip.get_indexer("broadcast")

## Similar to Xing

In [ ]:
flipped_values = [None for i in range(32)]

row_indexer_handle.set(1)
column_indexer_handle.set(1)

#chip.read_all_block("ETROC2", "Pixel Config")
# Flip all bits
chip.read_all_block('ETROC2', 'Pixel Config')
for i in range(32):
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
    value = int(handle.get(),0)
    flipped = value ^ 0xff
    #print(f'Register {i} was {value}, flipped to {flipped}')
    handle.set(hex(flipped))
    flipped_values[i] = flipped

#print(f'Values should be {flipped_values}')

broadcast_handle.set(True)
chip.write_all_block("ETROC2", "Pixel Config")

for row in range(16):
    for col in range(16):
        row_indexer_handle.set(row)
        column_indexer_handle.set(col)

        chip.read_all_block('ETROC2', 'Pixel Config')

        for i in range(32):
            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

            value = int(handle.get(),0)
            if value != flipped_values[i]:
                print('col: ', col, 'row: ', row, f', Pixel readback values do not match the default values! Value: {value} should be {flipped_values[i]}')

In [ ]:
for row in range(16):
    for col in range(16):
        for i in range(32):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

            value = int(handle.get(),0)
            flipped = value ^ 0xff
            #print(f'Register {i} was {value}, flipped to {flipped}')

            handle.set(hex(flipped))

            chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)
            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped:
                print('col: ', col, 'row: ', row, ', Pixel byte readback values do not match the default values!')

## Read modify write per block (pixel)

In [ ]:


for row in range(16):
    for col in range(16):
        row_indexer_handle.set(row)
        column_indexer_handle.set(col)

        chip.read_all_block('ETROC2', 'Pixel Config')

        for i in range(32):
            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
            value = int(handle.get(),0)
            flipped = value ^ 0xff
            #print(f'Register {i} was {value}, flipped to {flipped}')
            handle.set(hex(flipped))
            flipped_values[i] = flipped

        chip.write_all_block('ETROC2', 'Pixel Config')

        for i in range(32):
            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

            value = int(handle.get(),0)
            if value != flipped_values[i]:
                print('col: ', col, 'row: ', row, f', Pixel readback values do not match the default values! Value: {value} should be {flipped_values[i]}')

## Fixed Xing test

In [ ]:
row_indexer_handle.set(0)
column_indexer_handle.set(0)
chip.read_all_block('ETROC2', 'Pixel Config')

for i in range(32):
    row_indexer_handle.set(0)
    column_indexer_handle.set(0)
    
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    value = int(handle.get(),0)
    flipped = value ^ 0xff

    handle.set(hex(flipped))
    broadcast_handle.set(True)
    chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)

    failed = False
    for row in range(16):
        for col in range(16):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped:
                failed = True
                break
        if failed:
            break
    if failed:
        print(f"Broadcast failed for PixCfg{i}")

## Random order

In [ ]:
from numpy.random import permutation
row_indexer_handle.set(0)
column_indexer_handle.set(0)
chip.read_all_block('ETROC2', 'Pixel Config')

for i in permutation(32):
    row_indexer_handle.set(0)
    column_indexer_handle.set(0)
    
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    value = int(handle.get(),0)
    flipped = value ^ 0xff

    handle.set(hex(flipped))
    broadcast_handle.set(True)
    chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)

    failed = False
    for row in range(16):
        for col in range(16):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped:
                failed = True
                break
        if failed:
            break
    if failed:
        print(f"Broadcast failed for PixCfg{i}")

## Flip all first, then check

In [ ]:
from numpy.random import permutation
flipped_values = [None for i in range(32)]

b_row = 15  # This makes it always work
b_col = 15
#b_row = 1 # Only works depending on read
#b_col = 1
#b_row = 0 # Only works depending on read
#b_col = 0
row_indexer_handle.set(b_row)
column_indexer_handle.set(b_col)
chip.read_all_block('ETROC2', 'Pixel Config')

write_order = []
for i in permutation(32):
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
    write_order += [f"PixCfg{i}"]

    value = int(handle.get(),0)
    flipped = value ^ 0xff

    flipped_values[i] = flipped

    handle.set(hex(flipped))
    broadcast_handle.set(True)
    chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)  # So far, write check does not seem to make a difference
    #row_indexer_handle.set(0)  # Does not work for (0, 0)
    #column_indexer_handle.set(0)
    row_indexer_handle.set(1)  # Does not work for (0, 0)
    column_indexer_handle.set(1)
    #row_indexer_handle.set(2)  # Does not work for (0, 0)
    #column_indexer_handle.set(2)
    #row_indexer_handle.set(3)  # Does not work for (0, 0)
    #column_indexer_handle.set(3)
    #row_indexer_handle.set(4)  # Does not work for (0, 0)
    #column_indexer_handle.set(4)
    #row_indexer_handle.set(8)  # Does not work for (0, 0)
    #column_indexer_handle.set(8)
    #row_indexer_handle.set(0)  # Does not work for (0, 0)
    #column_indexer_handle.set(15)
    #row_indexer_handle.set(1)  # Does not work for (0, 0)
    #column_indexer_handle.set(15)
    #row_indexer_handle.set(8)  # Does not work for (0, 0)
    #column_indexer_handle.set(15)
    #row_indexer_handle.set(15)  # Does not work for (0, 0)
    #column_indexer_handle.set(0)
    #row_indexer_handle.set(15)  # Does not work for (0, 0)
    #column_indexer_handle.set(1)
    #row_indexer_handle.set(15)  # Does not work for (0, 0), (1, 1), works for (15, 15)
    #column_indexer_handle.set(8)
    #row_indexer_handle.set(14)  # Does not work for (1, 1), works for (15, 15)
    #column_indexer_handle.set(14)
    #row_indexer_handle.set(15)  # Works
    #column_indexer_handle.set(15)
    #chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
    row_indexer_handle.set(b_row)
    column_indexer_handle.set(b_col)
    #chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")

    #chip.read_all_block('ETROC2', 'Pixel Config')  # Does not work
    #sleep(1)  # No effect on breaking

print(f'Write order: {write_order}')
for i in range(32):
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    failed = False
    row_fail = None
    col_fail = None
    for row in range(16):
        for col in range(16):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped_values[i]:
                failed = True
                row_fail = row
                col_fail = col
                break
        if failed:
            break
    if failed:
        print(f"Broadcast failed for PixCfg{i} on row {row_fail} col {col_fail}")
        break

Write order: ['PixCfg6', 'PixCfg9', 'PixCfg3', 'PixCfg5', 'PixCfg4', 'PixCfg2', 'PixCfg27', 'PixCfg19', 'PixCfg10', 'PixCfg26', 'PixCfg23', 'PixCfg12', 'PixCfg17', 'PixCfg30', 'PixCfg11', 'PixCfg14', 'PixCfg18', 'PixCfg16', 'PixCfg7', 'PixCfg1', 'PixCfg22', 'PixCfg29', 'PixCfg24', 'PixCfg21', 'PixCfg20', 'PixCfg8', 'PixCfg25', 'PixCfg28', 'PixCfg31', 'PixCfg0', 'PixCfg13', 'PixCfg15']


## Our tests

In [ ]:
register_under_test = "disDataReadout"
register_under_test = "upperTOT"
register_under_test = "lowerTOA"
register_under_test = "upperTOA"
register_under_test = "upperCal"

row_indexer_handle.set(15)
column_indexer_handle.set(15)
chip.read_all_block('ETROC2', 'Pixel Config')

# for row in range(16):
#     for col in range(16):
#         row_indexer_handle.set(row)
#         column_indexer_handle.set(col)
#         retrievevalue = i2c_conn.pixel_decoded_register_read(register_under_test, "Config", chip)
#         origvalue = int(retrievevalue, 16)
#         print(f'Pixel {row_indexer_handle.get()}, {column_indexer_handle.get()}')
#         print(bin(origvalue))
#         binaryorig = bin(origvalue)[2:]
#         bitflipvalue = origvalue ^ int("0xfffffffff", 16)
#         new_value = hex(int("0b" + (bin(bitflipvalue)[2:])[-len(binaryorig):], 0))
#         print(f"Trying to set to: {bin(int(new_value, 0))}")
#         broadcast_handle,_,_ = chip.get_indexer("broadcast")
#         broadcast_handle.set(True)
#         handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", register_under_test)
#         handle.set(new_value)
#         chip.write_decoded_value("ETROC2", "Pixel Config", register_under_test)

handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", register_under_test)

retrievevalue = handle.get()
origvalue = int(retrievevalue, 0)
print(f'Pixel {row_indexer_handle.get()}, {column_indexer_handle.get()}')
print(bin(origvalue))
binaryorig = bin(origvalue)[2:]
bitflipvalue = origvalue ^ int("0xfffffffff", 16)
print("bitflipvalue:",bin(bitflipvalue))
new_value = hex(int("0b" + (bin(bitflipvalue)[2:])[-len(binaryorig):], 0))
print(f"Trying to set to: {bin(int(new_value, 0))}")

Pixel 15, 15
0b0
bitflipvalue: 0b111111111111111111111111111111111111
Trying to set to: 0b1


In [ ]:
handle.set(new_value)
broadcast_handle.set(True)
chip.write_decoded_value("ETROC2", "Pixel Config", register_under_test,write_check=False)

In [ ]:
for row in range(16):
    for col in range(16):
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)

        chip.read_decoded_value("ETROC2", "Pixel Config", register_under_test)
        handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", register_under_test)
        value = handle.get()

        if int(value,0) != int(new_value,0):
            print(f'Error on pixel ({row},{col}): {value} should be {new_value}')